##### Importing Important Libraries
###### 1. Request to get search results.<br>2. BeautifulSoup to scrap data.<br>3. Pandas data transformation.

In [ ]:
import requests as re
from bs4 import BeautifulSoup as bs
import pandas as pd

##### Get Data
###### We have to scrap from this site - https://www.architonic.com/en/accounts/representatives/10004/1<br>Headers is used to mimic a real browser to bypass scrap blocker or IP ban.<br>There are 3 pages containing result, so reading one by one and combined together.<br>If the Response code is 200, means the data read successful.

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
html = ""

#read all 3 pages
for page in range(1, 4):
    url = f"https://www.architonic.com/en/accounts/representatives/10004/{page}"
    response = re.get(url,headers=headers)
    
    if response.status_code == 200:
        html += response.text  #Append the page HTML content
    else:
        print(f"Failed to fetch page {page}")
print("All pages read!")

##### Extract data
###### If incase we request web multiple times, IP might get banned. So, i've extracted the data and exported in .html file so that we can perform data scraping without further issues.

In [ ]:
with open("output.html","w",encoding="utf-8") as f:
    f.write(html)
    print("Extract and Write Successful!")

##### Create soup object with BeautifulSoup

###### We create an object from an HTML file for easy parsing and manipulation.

In [ ]:
with open("output.html","r") as f:
    soup=bs(f.read(),'html.parser')
print(soup.prettify())

##### Scrap Data
###### Using soup object, extracting required data like Profile Group, Profile and Location.<br>In Location, State and Country were kept together. So it has been separated in different column.<br>The extracted data is kept in dictionary.

In [ ]:
data={"Profile Group":[],"Profile":[],"State":[],"Country":[]}

#Profile Group
pg=soup.find_all(class_="account-type small-12 columns")
for i in pg:
    data["Profile Group"].append(i.get_text())

#Profile
pf=soup.find_all(class_="account-title small-12 columns")
for i in pf:
    data["Profile"].append(i.get_text())

#Location
loc=soup.find_all(class_="account-info small-12 columns")
for i in loc:
    ad=i.get_text().strip().split(", ")
    data["State"].append(ad[0])
    data["Country"].append(ad[1])

##### Export Data
###### The data in dict is transformed to pandas DataFrame and exported in excel format.

In [ ]:
df=pd.DataFrame.from_dict(data)
df.to_excel("Output.xlsx",index=False)